[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/siowcm/cancer/blob/main/code/02-cancer.ipynb)

# Import libraries

In [1]:
# import libraries
import glob
import os
import re
import shutil
from imutils import paths
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
# clone github repo
!git clone https://github.com/siowcm/cancer.git

In [ ]:
%cd ./cancer/code

In [2]:
df = pd.read_csv("../data/cancer.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277524 entries, 0 to 277523
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   patient_id  277524 non-null  int64 
 1   label       277524 non-null  int64 
 2   path        277524 non-null  object
 3   file_name   277524 non-null  object
 4   img_x       277524 non-null  int64 
 5   img_y       277524 non-null  int64 
 6   height      277524 non-null  int64 
 7   width       277524 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 16.9+ MB


# Preprocessing

In [4]:
#parameters
TRAIN_PATH = "../data/train"
VAL_PATH = "../data/val"
TEST_PATH = "../data/test"
IMG_SIZE = (48, 48)
BATCH_SIZE = 32
PATIENCE = 10
NUM_EPOCHS = 50
INIT_LR = 0.001

In [5]:
# create training, validation and testing data
df_train_and_val, df_test = train_test_split(
    df,
    train_size=0.8,
    random_state=42,
    stratify=df["label"]
)

df_train, df_val = train_test_split(
    df_train_and_val,
    train_size=0.8,
    random_state=42,
    stratify=df_train_and_val["label"]
)

In [6]:
# check the shape of training and testing data
print(f"Shape of training df: {df_train.shape}")
print(f"Shape of validation df: {df_val.shape}")
print(f"Shape of testing df: {df_test.shape}")

Shape of training df: (177615, 8)
Shape of validation df: (44404, 8)
Shape of testing df: (55505, 8)


In [7]:
# create folder/ subfolders if it does not exist
for folder in [TRAIN_PATH, VAL_PATH, TEST_PATH]:
    if not os.path.exists(folder):
        os.makedirs(folder)
    if not os.path.exists(os.path.join(folder, "0")):
        os.makedirs(os.path.join(folder, "0"))
    if not os.path.exists(os.path.join(folder, "1")):
        os.makedirs(os.path.join(folder, "1"))

In [8]:
#copy to newly created folders
for index, data in df_train.iterrows():
    shutil.copy2(
        data["path"], os.path.join(TRAIN_PATH, str(data["label"]))
    )
    
for index, data in df_val.iterrows():
    shutil.copy2(
        data["path"], os.path.join(VAL_PATH, str(data["label"]))
    )
    
for index, data in df_test.iterrows():
    shutil.copy2(
        data["path"], os.path.join(TEST_PATH, str(data["label"]))
    )

In [9]:
# check folder structure
!apt-get install tree
!cd ../data ; tree --dirsfirst --filelimit 5

.
├── archive  [280 entries exceeds filelimit, not opening dir]
├── test
│   ├── 0  [39748 entries exceeds filelimit, not opening dir]
│   └── 1  [15757 entries exceeds filelimit, not opening dir]
├── train
│   ├── 0  [127192 entries exceeds filelimit, not opening dir]
│   └── 1  [50423 entries exceeds filelimit, not opening dir]
├── val
│   ├── 0  [31798 entries exceeds filelimit, not opening dir]
│   └── 1  [12606 entries exceeds filelimit, not opening dir]
└── cancer.csv

10 directories, 1 file


In [10]:
#function to load image
def image_loader(path):
    image = tf.io.read_file(path)                                # read image file as binary
    image = tf.image.decode_png(image, channels=3)               # decode into image tensor
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)# converts to float32 data type and scaling the values appropriately before casting.
    image = tf.image.resize(image, IMG_SIZE)                     # resize the image
    label = tf.strings.split(path, os.path.sep)[-2]              # parse the class label from the file path
    label = tf.strings.to_number(label, tf.int32)
    return (image, label)

In [27]:
#function to augment images
@tf.function                                        #decarator code to create Python-independent dataflow graphs out of your Python code. 
def augmentation(image, label):                     #This will help you create performant and portable model.
    image = tf.image.random_flip_left_right(image)  #random horizontal flips
    image = tf.image.random_flip_up_down(image)     #random horizontal flips
    return (image, label)                           #more augmenetation from tensorflow API https://www.tensorflow.org/api_docs/python/tf/image

In [12]:
#create list pf images paths
train_paths = list(paths.list_images(TRAIN_PATH))
val_paths = list(paths.list_images(VAL_PATH))
test_paths = list(paths.list_images(TEST_PATH))

#print output for clarity 
print(train_paths[:2])
print(val_paths[:2])
print(test_paths[:2])

['../data/train/0/14079_idx5_x2151_y1401_class0.png', '../data/train/0/14157_idx5_x1651_y301_class0.png']
['../data/val/0/15510_idx5_x1801_y1001_class0.png', '../data/val/0/10295_idx5_x801_y951_class0.png']
['../data/test/0/12749_idx5_x1451_y701_class0.png', '../data/test/0/8867_idx5_x2501_y1301_class0.png']


In [13]:
#create class weights
train_labels = [int(p.split(os.path.sep)[-2]) for p in train_paths]
train_labels = keras.utils.to_categorical(train_labels)              #one-hot encode list into arrary 
class_totals = train_labels.sum(axis=0)
class_weight = {}
for i in range(0, len(class_totals)):
    class_weight[i] = class_totals.max() / class_totals[i]
print(f"class weights: {class_weight}")

class weights: {0: 1.0, 1: 2.5224996}


In [65]:
'''
Consider using this block of code for more fancify augmentatin from keras API
https://keras.io/api/layers/preprocessing_layers/image_augmentation/
'''

# data_augmentation = tf.keras.Sequential(
#     [
#         keras.layers.RandomCrop(height, width, seed=None, **kwargs),
#         keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=None, **kwargs)
#         keras.layers.RandomTranslation()
#.        keras.layers.Rotation(),
#         keras.layers.Zoom(),
#         keras.layers.Contrast()
#     ]
# )

# train_data = tf.data.Dataset.from_tensor_slices(train_paths)
# train_data = (                         
#     train_data.shuffle(len(train_paths))                                               # shuffle data, optional as already did train_test_split in scikit-learn
#     .map(image_loader, num_parallel_calls=tf.data.AUTOTUNE)                            # load the image
#     .map(augmentation, num_parallel_calls=tf.data.AUTOTUNE)                            # apply augmentation
#     .map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE)  # augementation via Keras API}
#     .cache()                                                                           # caching for fast read
#     .batch(batch_size=BATCH_SIZE)                                                      # batching the data
#     .prefetch(tf.data.AUTOTUNE)                                                        # allows later elements to be prepared while the current element is being processed
# )

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(train_paths)
train_data = (
    train_data.shuffle(len(train_paths))                     # shuffle data, optional as already did train_test_split in scikit-learn
    .map(image_loader, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .map(augmentation, num_parallel_calls=tf.data.AUTOTUNE)  # apply augmentation
    .cache()                                                 # caching for fast read
    .batch(batch_size=BATCH_SIZE)                            # batching the data
    .prefetch(tf.data.AUTOTUNE)                              # allows later elements to be prepared while the current element is being processed
)


val_data = tf.data.Dataset.from_tensor_slices(val_paths)              #no augmentation & shuffle required for test data as it is ground truth
val_data = (
    val_data.map(image_loader, num_parallel_calls=tf.data.AUTOTUNE)   # load the image
    .batch(batch_size=BATCH_SIZE)                                     # batching the data
    .prefetch(tf.data.AUTOTUNE)                                       # allows later elements to be prepared while the current element is being processed
)


test_data = tf.data.Dataset.from_tensor_slices(train_paths)           #no augmentation & shuffle required for test data as it is ground truth
test_data = (
    test_data.map(image_loader, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .batch(batch_size=BATCH_SIZE)                                     # batching the data
    .prefetch(tf.data.AUTOTUNE)                                       # allows later elements to be prepared while the current element is being processed
)

# Simple CNN Model

In [ ]:
model_cnn = keras.models.Sequential(
    [
        keras.layers.Conv2D(
            filters=10,
            kernel_size=3,
            activation="relu",
            input_shape=(48, 48, 3),
        ),
        keras.layers.Conv2D(filters=10, kernel_size=3, activation="relu"),
        keras.layers.MaxPool2D(pool_size=2, padding="valid"),
        keras.layers.Conv2D(filters=10, kernel_size=3, activation="relu"),
        keras.layers.Conv2D(filters=10, kernel_size=3, activation="relu"),
        keras.layers.MaxPool2D(pool_size=2, padding="valid"),
        keras.layers.Flatten(),
        keras.layers.Dense(units=1, activation="sigmoid"),
    ]
)

In [ ]:
model_cnn.summary()

In [ ]:
def compile_fit_model(model, save_path):
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=INIT_LR),
        metrics=[
            keras.metrics.BinaryAccuracy(),
            keras.metrics.AUC(),
            keras.metrics.Precision(),
            keras.metrics.Recall(),
        ],
    )

    callbacks_list = [
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=PATIENCE,
            restore_best_weights=True
        ),
        keras.callbacks.ModelCheckpoint(filepath=save_path, save_best_only=True),
        keras.callbacks.ReduceLROnPlateau(),
    ]

    history = model.fit(
        x=train_data,
        epochs=NUM_EPOCHS,
        callbacks= callbacks_list,
        validation_data=val_data,
        class_weight=class_weight
    )
    
    return history, model

In [ ]:
history_cnn, model_cnn = compile_fit_model(model=model_cnn, save_path="cnn.h5")

In [ ]:
model_cnn.evaluate(test_data)

In [ ]:
result_cnn = pd.DataFrame(history_cnn.history)

In [ ]:
result_cnn.plot()

# Transfer Learning

In this section, the models will be built by using the convolution network of the pre-trained model (e.g. VGG16) as a backbone to extract features and add a fully-connected layer. 

## VGG16

In [ ]:
#function to load image
def image_loader_vgg16(path):
    image = tf.io.read_file(path)                                # read image file as binary
    image = tf.image.decode_png(image, channels=3)               # decode into image tensor
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)# converts to float32 data type and scaling the values appropriately before casting.
    image = tf.image.resize(image, IMG_SIZE)                     # resize the image
    image = keras.applications.vgg16.preprocess_input(image)     # custom preprocessing for VGG16
    label = tf.strings.split(path, os.path.sep)[-2]              # parse the class label from the file path
    label = tf.strings.to_number(label, tf.int32)
    return (image, label)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(train_paths)
train_data = (
    train_data.shuffle(len(train_paths))                           # shuffle data, optional as already did train_test_split in scikit-learn
    .map(image_loader_vgg16, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .map(augmentation, num_parallel_calls=tf.data.AUTOTUNE)        # apply augmentation
    .cache()                                                       # caching for fast read
    .batch(batch_size=BATCH_SIZE)                                  # batching the data
    .prefetch(tf.data.AUTOTUNE)                                    # allows later elements to be prepared while the current element is being processed
)


val_data = tf.data.Dataset.from_tensor_slices(val_paths)                    #no augmentation & shuffle required for test data as it is ground truth
val_data = (
    val_data.map(image_loader_vgg16, num_parallel_calls=tf.data.AUTOTUNE)   # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)


test_data = tf.data.Dataset.from_tensor_slices(train_paths)                 #no augmentation & shuffle required for test data as it is ground truth
test_data = (
    test_data.map(image_loader_vgg16, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)

In [ ]:
VGG16 = keras.applications.VGG16(include_top=False,weights="imagenet")
VGG16.trainable = False

In [ ]:
model_vgg16 = keras.models.Sequential(
    [
        VGG16,
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=1, activation='sigmoid')

    ]
)

In [ ]:
model_vgg16.summary()

In [ ]:
history_vgg16, model_vgg16 = compile_fit_model(model=model_vgg16, save_path="vgg16.h5")

In [ ]:
model_vgg16.evaluate(test_data)

In [ ]:
result_vgg16 = pd.DataFrame(history_vgg16.history)

In [ ]:
result_vgg16.plot()

## EfficientNetB0

In [ ]:
#function to load image
def image_loader_effnetb0(path):
    image = tf.io.read_file(path)                                # read image file as binary
    image = tf.image.decode_png(image, channels=3)               # decode into image tensor
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)# converts to float32 data type and scaling the values appropriately before casting.
    image = tf.image.resize(image, IMG_SIZE)                     # resize the image
    image = keras.applications.efficientnet.preprocess_input(image)     # custom preprocessing for efficientnet
    label = tf.strings.split(path, os.path.sep)[-2]              # parse the class label from the file path
    label = tf.strings.to_number(label, tf.int32)
    return (image, label)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(train_paths)
train_data = (
    train_data.shuffle(len(train_paths))                           # shuffle data, optional as already did train_test_split in scikit-learn
    .map(image_loader_effnetb0, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .map(augmentation, num_parallel_calls=tf.data.AUTOTUNE)        # apply augmentation
    .cache()                                                       # caching for fast read
    .batch(batch_size=BATCH_SIZE)                                  # batching the data
    .prefetch(tf.data.AUTOTUNE)                                    # allows later elements to be prepared while the current element is being processed
)


val_data = tf.data.Dataset.from_tensor_slices(val_paths)                    #no augmentation & shuffle required for test data as it is ground truth
val_data = (
    val_data.map(image_loader_effnetb0, num_parallel_calls=tf.data.AUTOTUNE)   # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)


test_data = tf.data.Dataset.from_tensor_slices(train_paths)                 #no augmentation & shuffle required for test data as it is ground truth
test_data = (
    test_data.map(image_loader_effnetb0, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)

In [ ]:
effnetb0 = keras.applications.EfficientNetB0(include_top=False,weights="imagenet")
effnetb0.trainable = False

In [ ]:
model_effnetb0 = keras.models.Sequential(
    [
        effnetb0,
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=1, activation='sigmoid')

    ]
)

In [ ]:
model_effnetb0.summary()

In [ ]:
history_effnetb0, model_effnetb0 = compile_fit_model(model=model_effnetb0, save_path="effnetb0.h5")

In [ ]:
model_effnetb0.evaluate(test_data)

In [ ]:
result_effnetb0 = pd.DataFrame(history_effnetb0.history)

In [ ]:
result_effnetb0.plot()

## Xception

In [ ]:
#function to load image
def image_loader_xception(path):
    image = tf.io.read_file(path)                                # read image file as binary
    image = tf.image.decode_png(image, channels=3)               # decode into image tensor
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)# converts to float32 data type and scaling the values appropriately before casting.
    image = tf.image.resize(image, IMG_SIZE)                     # resize the image
    image = keras.applications.xception.preprocess_input(image)     # custom preprocessing for xception
    label = tf.strings.split(path, os.path.sep)[-2]              # parse the class label from the file path
    label = tf.strings.to_number(label, tf.int32)
    return (image, label)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(train_paths)
train_data = (
    train_data.shuffle(len(train_paths))                           # shuffle data, optional as already did train_test_split in scikit-learn
    .map(image_loader_xception, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .map(augmentation, num_parallel_calls=tf.data.AUTOTUNE)        # apply augmentation
    .cache()                                                       # caching for fast read
    .batch(batch_size=BATCH_SIZE)                                  # batching the data
    .prefetch(tf.data.AUTOTUNE)                                    # allows later elements to be prepared while the current element is being processed
)


val_data = tf.data.Dataset.from_tensor_slices(val_paths)                    #no augmentation & shuffle required for test data as it is ground truth
val_data = (
    val_data.map(image_loader_xception, num_parallel_calls=tf.data.AUTOTUNE)   # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)


test_data = tf.data.Dataset.from_tensor_slices(train_paths)                 #no augmentation & shuffle required for test data as it is ground truth
test_data = (
    test_data.map(image_loader_xception, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)

In [ ]:
xception = keras.applications.Xception(include_top=False,weights="imagenet")
xception.trainable = False

In [ ]:
model_xception = keras.models.Sequential(
    [
        xception,
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=1, activation='sigmoid')

    ]
)

In [ ]:
model_xception.summary()

In [ ]:
history_xception, model_xception = compile_fit_model(model=model_xception, save_path="xception.h5")

In [ ]:
model_xception.evaluate(test_data)

In [ ]:
result_xception = pd.DataFrame(history_xception.history)

In [ ]:
result_xception.plot()

## DenseNet121

In [ ]:
#function to load image
def image_loader_densenet121(path):
    image = tf.io.read_file(path)                                # read image file as binary
    image = tf.image.decode_png(image, channels=3)               # decode into image tensor
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)# converts to float32 data type and scaling the values appropriately before casting.
    image = tf.image.resize(image, IMG_SIZE)                     # resize the image
    image = keras.applications.densenet.preprocess_input(image)     # custom preprocessing for densenet121
    label = tf.strings.split(path, os.path.sep)[-2]              # parse the class label from the file path
    label = tf.strings.to_number(label, tf.int32)
    return (image, label)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(train_paths)
train_data = (
    train_data.shuffle(len(train_paths))                           # shuffle data, optional as already did train_test_split in scikit-learn
    .map(image_loader_densenet121, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .map(augmentation, num_parallel_calls=tf.data.AUTOTUNE)        # apply augmentation
    .cache()                                                       # caching for fast read
    .batch(batch_size=BATCH_SIZE)                                  # batching the data
    .prefetch(tf.data.AUTOTUNE)                                    # allows later elements to be prepared while the current element is being processed
)


val_data = tf.data.Dataset.from_tensor_slices(val_paths)                    #no augmentation & shuffle required for test data as it is ground truth
val_data = (
    val_data.map(image_loader_densenet121, num_parallel_calls=tf.data.AUTOTUNE)   # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)


test_data = tf.data.Dataset.from_tensor_slices(train_paths)                 #no augmentation & shuffle required for test data as it is ground truth
test_data = (
    test_data.map(image_loader_densenet121, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)

In [ ]:
densenet121 = keras.applications.DenseNet121(include_top=False,weights="imagenet")
densenet121.trainable = False

In [ ]:
model_densenet121 = keras.models.Sequential(
    [
        densenet121,
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=1, activation='sigmoid')

    ]
)

In [ ]:
model_densenet121.summary()

In [ ]:
history_densenet121, model_densenet121 = compile_fit_model(model=model_densenet121, save_path="densenet152.h5")

In [ ]:
model_densenet121.evaluate(test_data)

In [ ]:
result_densenet121 = pd.DataFrame(history_densenet121.history)

In [ ]:
result_densenet121.plot()

## ResNet152V2 

In [ ]:
#function to load image
def image_loader_resnet152(path):
    image = tf.io.read_file(path)                                # read image file as binary
    image = tf.image.decode_png(image, channels=3)               # decode into image tensor
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)# converts to float32 data type and scaling the values appropriately before casting.
    image = tf.image.resize(image, IMG_SIZE)                     # resize the image
    image = keras.applications.resnet_v2.preprocess_input(image)     # custom preprocessing for resnet152
    label = tf.strings.split(path, os.path.sep)[-2]              # parse the class label from the file path
    label = tf.strings.to_number(label, tf.int32)
    return (image, label)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(train_paths)
train_data = (
    train_data.shuffle(len(train_paths))                           # shuffle data, optional as already did train_test_split in scikit-learn
    .map(image_loader_resnet152, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .map(augmentation, num_parallel_calls=tf.data.AUTOTUNE)        # apply augmentation
    .cache()                                                       # caching for fast read
    .batch(batch_size=BATCH_SIZE)                                  # batching the data
    .prefetch(tf.data.AUTOTUNE)                                    # allows later elements to be prepared while the current element is being processed
)


val_data = tf.data.Dataset.from_tensor_slices(val_paths)                    #no augmentation & shuffle required for test data as it is ground truth
val_data = (
    val_data.map(image_loader_resnet152, num_parallel_calls=tf.data.AUTOTUNE)   # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)


test_data = tf.data.Dataset.from_tensor_slices(train_paths)                 #no augmentation & shuffle required for test data as it is ground truth
test_data = (
    test_data.map(image_loader_resnet152, num_parallel_calls=tf.data.AUTOTUNE)  # load the image
    .batch(batch_size=BATCH_SIZE)                                           # batching the data
    .prefetch(tf.data.AUTOTUNE)                                             # allows later elements to be prepared while the current element is being processed
)

In [ ]:
resnet152 = keras.applications.ResNet152V2(include_top=False,weights="imagenet")
resnet152.trainable = False

In [ ]:
model_resnet152 = keras.models.Sequential(
    [
        resnet152,
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=1, activation='sigmoid')

    ]
)

In [ ]:
model_resnet152.summary()

In [ ]:
history_resnet152, model_resnet152 = compile_fit_model(model=model_resnet152, save_path="resnet152.h5")

In [ ]:
model_resnet152.evaluate(test_data)

In [ ]:
result_resnet152 = pd.DataFrame(history_resnet152.history)

In [ ]:
result_resnet152.plot()